In [119]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta
from utils import format_number
import numpy as np

In [120]:
# set parameters
LAST_N_DAYS = 30
INCENTIVE_PROGRAMS_ONLY = False#True

# Incentive Program Summary
Status of programs live, completed and to be announced by season.

In [121]:
df_info = pd.read_csv("inputs/" + "op_incentive_program_info" + ".csv")

if INCENTIVE_PROGRAMS_ONLY:
    df_info = df_info[df_info["Incentive / Growth Program Included?"] == "Yes"]

In [122]:
# convert to datetime
df_info["announced_date"] = pd.to_datetime( df_info["Announced On"] )
df_info["End Date"] = pd.to_datetime( df_info["End Date"] )

# convert program status into ordered categorical type
cat_size_order = CategoricalDtype(
    ["Live ‎🔥", "Coming soon ‎⏳", "Completed"], 
    ordered=True
)
df_info["Status"] = df_info["Status"].astype(cat_size_order)

for i in ['GovFund','GovFund Growth Experiments','All Programs']:
    # Assign the filters
    if i == 'GovFund':
        filter_name = " - GovFund Only"
        df_choice = df_info[df_info['Source'] != 'Partner Fund'].copy()
    elif i == 'GovFund Growth Experiments':
        filter_name = " - GovFund Growth Exp."
        df_choice = df_info[df_info['Source'] != 'Partner Fund'].copy()
        df_choice =  df_choice[df_choice['Incentive / Growth Program Included?'] == 'Yes']
    else:
        filter_name = ""
        df_choice = df_info.copy()

    # clean up for columns needed
    df_choice = df_choice[["Source","Status","# OP Allocated","App Name","announced_date", "End Date"]]
    summary = pd.pivot_table(df_choice, values=["# OP Allocated", "App Name"], index=["Status", "Source"], \
        aggfunc={"# OP Allocated":"sum", "App Name":"count"})

    subtotal_name = "Subtotal" + filter_name
    # calculate subtotals on program status
    result=pd.concat([summary,summary.groupby(level=0).sum().assign(item_name=subtotal_name).set_index("item_name",append=True)]).sort_index(level=[0,1])
    result = result.sort_index(level=[0, 1], ascending=[True, False])

    # add grand total to summary
    result.loc[("Grand Total"), "# OP Allocated"] = summary["# OP Allocated"].sum()
    result.loc[("Grand Total"), "App Name"] = summary["App Name"].sum()

    # cleanup display
    result["# Programs"] = result["App Name"].astype(int)
    result["# OP Allocated (M)"] = result["# OP Allocated"].apply(format_number)

    # calculate percentage of total
    result.loc[(slice(None), subtotal_name), "# OP Allocated"] / summary["# OP Allocated"].sum()
    result["% OP Allocated"] = round(result.loc[(slice(None), subtotal_name), "# OP Allocated"] / summary["# OP Allocated"].sum() * 100).\
        astype(str).replace("\.0", "", regex=True) + "%"
    result["% OP Allocated"].fillna("-", inplace=True)

    result = result.replace((0, "0.0M"), "-")
    display(result.drop(columns=["# OP Allocated", "App Name"]))

# Programs # OP Allocated (M)  \
Status         Source                                                  
Live ‎🔥        Subtotal - GovFund Only         36              42.5M   
               Governance - Season 3            -                0.0   
               Governance - Season 2           11               7.3M   
               Governance - Season 1            9               4.5M   
               Governance - Phase 0            16              30.7M   
Coming soon ‎⏳ Subtotal - GovFund Only         32              10.9M   
               Governance - Season 3           12               2.1M   
               Governance - Season 2           13               5.2M   
               Governance - Season 1            4               1.3M   
               Governance - Phase 0             3               2.2M   
Completed      Subtotal - GovFund Only          2               1.2M   
               Governance - Season 3            -                0.0   
               Governance - Season 2            -                0.0   
               Governance - Season 1            -                0.0   
               Governance - Phase 0             2               1.2M   
Grand Total                                    70              54.6M   

                                       % OP Allocated  
Status         Source                                  
Live ‎🔥        Subtotal - GovFund Only            78%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Coming soon ‎⏳ Subtotal - GovFund Only            20%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Completed      Subtotal - GovFund Only             2%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Grand Total                                         -

,App Name,# OP Allocated,Actions Rewarded,Announced On,Announcement Link,App Link,App Name Map Override,App Type,Created time,Description,End Date,Gov Proposal Link,Incentive / Growth Program Included?,Source,Start Date,Status,Status - Season,announced_date


# Programs # OP Allocated (M)  \
Status         Source                                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.         36              42.5M   
               Governance - Season 3                   -                0.0   
               Governance - Season 2                  11               7.3M   
               Governance - Season 1                   9               4.5M   
               Governance - Phase 0                   16              30.7M   
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.         32              10.9M   
               Governance - Season 3                  12               2.1M   
               Governance - Season 2                  13               5.2M   
               Governance - Season 1                   4               1.3M   
               Governance - Phase 0                    3               2.2M   
Completed      Subtotal - GovFund Growth Exp.          2               1.2M   
               Governance - Season 3                   -                0.0   
               Governance - Season 2                   -                0.0   
               Governance - Season 1                   -                0.0   
               Governance - Phase 0                    2               1.2M   
Grand Total                                           70              54.6M   

                                              % OP Allocated  
Status         Source                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.            78%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.            20%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Completed      Subtotal - GovFund Growth Exp.             2%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Grand Total                                                -

# Programs # OP Allocated (M)  \
Status         Source                                                
Live ‎🔥        Subtotal                      36              42.5M   
               Partner Fund                   -                0.0   
               Governance - Season 3          -                0.0   
               Governance - Season 2         11               7.3M   
               Governance - Season 1          9               4.5M   
               Governance - Phase 0          16              30.7M   
Coming soon ‎⏳ Subtotal                      32              10.9M   
               Partner Fund                   -                0.0   
               Governance - Season 3         12               2.1M   
               Governance - Season 2         13               5.2M   
               Governance - Season 1          4               1.3M   
               Governance - Phase 0           3               2.2M   
Completed      Subtotal                       5               9.7M   
               Partner Fund                   3               8.4M   
               Governance - Season 3          -                0.0   
               Governance - Season 2          -                0.0   
               Governance - Season 1          -                0.0   
               Governance - Phase 0           2               1.2M   
Grand Total                                  73              63.0M   

                                     % OP Allocated  
Status         Source                                
Live ‎🔥        Subtotal                         67%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Coming soon ‎⏳ Subtotal                         17%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Completed      Subtotal                         15%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Grand Total                                       -

In [123]:
# display new programs in last 30 days
df_new_programs = df_info[df_info["announced_date"]>pd.Timestamp("today") - timedelta(days = LAST_N_DAYS)].sort_values(by="announced_date", ascending=False)
if not df_new_programs.empty:
    df_new_programs["End Date"].fillna("-", inplace=True)
    display(df_new_programs)

,App Name,# OP Allocated,Actions Rewarded,Announced On,Announcement Link,App Link,App Name Map Override,App Type,Created time,Description,End Date,Gov Proposal Link,Incentive / Growth Program Included?,Source,Start Date,Status,Status - Season,announced_date
81,Sushiswap,504000.0,NaN,"March 3, 2023",https://twitter.com/SushiSwap/status/163167254...,NaN,Sushi,DEX (Decentralized Exchange),"November 28, 2022 10:10 AM",NaN,-,https://snapshot.org/#/opcollective.eth/propos...,Yes,Governance - Season 2,"March 3, 2023",Live ‎🔥,Live ‎🔥 - Governance - Season 2,2023-03-03
8,Angle,250000.0,NaN,"February 20, 2023",https://twitter.com/AngleProtocol/status/16280...,NaN,NaN,Stablecoin,"November 28, 2022 10:10 AM",NaN,-,https://snapshot.org/#/opcollective.eth/propos...,Yes,Governance - Season 2,"February 20, 2023",Live ‎🔥,Live ‎🔥 - Governance - Season 2,2023-02-20
93,WardenSwap,300000.0,NaN,"February 17, 2023",https://twitter.com/WardenSwap/status/16265929...,https://www.wardenswap.finance/#,NaN,DEX (Decentralized Exchange),"July 20, 2022 9:25 PM",NaN,-,https://gov.optimism.io/t/ready-gf-phase-1-pro...,Yes,Governance - Season 1,NaN,Live ‎🔥,Live ‎🔥 - Governance - Season 1,2023-02-17
68,Rainbow Wallet,420069.0,Trades/Swaps,"February 13, 2023",https://twitter.com/rainbowdotme/status/162514...,NaN,Rainbow,"DEX (Decentralized Exchange), Wallet / Tracker","November 28, 2022 10:10 AM",NaN,-,https://snapshot.org/#/opcollective.eth/propos...,Yes,Governance - Season 2,NaN,Live ‎🔥,Live ‎🔥 - Governance - Season 2,2023-02-13


In [124]:
# display completed programs in last 30 days
df_completed = df_info[(df_info['Status'] == 'Completed') & (df_info['End Date']>pd.Timestamp('today') - timedelta(days = LAST_N_DAYS))]\
        .sort_values(by='announced_date', ascending=False)
if not df_completed.empty:
        display(df_completed)